In [1]:
import os
os.chdir("../")
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
from gpytorch.kernels import RBFKernel, ScaleKernel, PeriodicKernel, MaternKernel, CosineKernel
from skgpytorch.models import SVGPRegressor, SGPRegressor
from scipy.stats import kurtosis,skew

In [3]:
import jax.numpy as jnp
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from utilities.fits import fit
# from utilities import plot
import tensorflow_probability.substrates.jax as tfp
dist = tfp.distributions
import pandas as pd
import jax.numpy as jnp
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from gpytorch.constraints import GreaterThan
from skgpytorch.metrics import mean_squared_error, negative_log_predictive_density
import time
import numpy as np

# from datasets.dataset_load import dataset_loader
from utilities import plot,fits,gmm,errors,predict,preprocess

# device = "cpu"
# torch.set_default_dtype(torch.float32)
# torch.set_default_tensor_type(torch.FloatTensor)

import flax.linen as nn
import jax.numpy as jnp

In [4]:
train ={1:{
            'start_time': "2011-04-28" ,
            'end_time': "2011-05-15"
        }, 
        3: {
        'start_time': "2011-04-19" ,
        'end_time': "2011-05-22"
        },
            
        }

test = {  2: {
                    'start_time': "2011-04-21" ,
                    'end_time': "2011-05-21"
                },
                }

# 6: {
#                     'start_time': "2011-05-25" ,
#                     'end_time': "2011-06-13"
#                 }

# 5: {
#                     'start_time': "2011-04-22" ,
#                     'end_time': "2011-06-01"
#                 }
appliances = ["Microwave", "Refrigerator",  "Dish Washer"] #

In [5]:
import pandas as pd
import jax.numpy as jnp
from datetime import datetime
from sklearn.preprocessing import StandardScaler


def dataset_load(appliances, train, test=None):
    x_train = []
    y_train = []
    x_train_timestamp = []
    x_train_mean = []
    x_train_std = []
    x_train_max_min = []
    x_train_main = []
    x_train_main_dif = []
    x_train_max = []
    x_train_min = []
    x_train_kurtosis = []
    x_train_skew = []
    n = 99
    m = 9
    units_to_pad = n // 2
    units_to_pad1 = m//2
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
    scaler_time = StandardScaler()
    scaler_mean = StandardScaler()
    scaler_std = StandardScaler()
    scaler_max_min = StandardScaler()
    scaler_main = StandardScaler()
    scaler_diff = StandardScaler()
    scaler_max = StandardScaler()
    scaler_min = StandardScaler()
    scaler_kurtosis = StandardScaler()
    scaler_skew = StandardScaler()
    # train
    for key, values in train.items():
        for app in range(len(appliances)):
            df = pd.read_csv(
                f"datasets/Building{key}_NILM_data_basic.csv", usecols=["Timestamp", "main", appliances[app]])
            df["date"] = pd.to_datetime(df["Timestamp"]).dt.date
            startDate = datetime.strptime(values["start_time"], "%Y-%m-%d").date()
            endDate = datetime.strptime(values["end_time"], "%Y-%m-%d").date()

            if startDate > endDate:
                raise "Start Date must be smaller than Enddate."

            df = df[(df["date"] >= startDate) & (df["date"] <= endDate)]
            df.dropna(inplace=True)
            # x = df["main"].values

            # x_diff = x
            # for i in range(1,len(x)):
            #     x_diff[i] = x_diff[i] - x_diff[i-1]
            if app == 0:
                x = df[appliances[app]].values
            else:
                print(app)
                x += df[appliances[app]].values
            if appliances[app] == "Refrigerator":
                y = df[appliances[app]].values
            # 
            # y = df[appliances[0]].values
        x_train_main.extend(x)
        timestamp_train = (pd.to_datetime(df["Timestamp"]).astype(int)/ 10**18).values
        x2 = x
        x = jnp.pad(x, (units_to_pad, units_to_pad),
                    'constant', constant_values=(0, 0))
        x = jnp.array([x[i: i + n] for i in range(len(x) - n + 1)])
        # print(0)
        x_train_mean.extend(jnp.mean(x, axis=1))
        # print(0)
        x_train_std.extend(jnp.std(x, axis=1))
        x1 = jnp.pad(x2, (units_to_pad1, units_to_pad1),
                    'constant', constant_values=(0, 0))
        x1 = jnp.array([x1[i: i + m] for i in range(len(x1) - m + 1)])
        x_train_max_min.extend(jnp.max(x, axis=1)-jnp.min(x,axis=1))
        x_train_max.extend(jnp.max(x1, axis = 1))
        x_train_min.extend(jnp.min(x1, axis = 1))
        x_train_kurtosis.extend(kurtosis(x,axis=1))
        x_train_skew.extend(skew(x,axis = 1))
        x_train.extend(x)
        y_train.extend(y)
        x_train_timestamp.extend(torch.tensor(timestamp_train))
        # x_train_diff.extend(x_diff)

    # for i in range(1,len(x_train)):
    #     x_train[i] = x_train[i] - x_train[i-1]
    x_train = jnp.array(x_train)
    y_train = jnp.array(y_train).reshape(-1, 1)
    x_train_timestamp = torch.tensor(x_train_timestamp).reshape(-1,1)
    x_train_main = jnp.array(x_train_main).reshape(-1,1)
    x_train_mean = jnp.array(x_train_mean).reshape(-1,1)
    x_train_std =  jnp.array(x_train_std).reshape(-1,1)
    x_train_max_min =  jnp.array(x_train_max_min).reshape(-1,1)
    x_train_max =  jnp.array(x_train_max).reshape(-1,1)
    x_train_skew = jnp.array(x_train_skew).reshape(-1,1)
    x_train_min =  jnp.array(x_train_min).reshape(-1,1)
    x_train_kurtosis = jnp.array(x_train_kurtosis).reshape(-1,1)
    # x_train_diff =  jnp.array(x_train_diff).reshape(-1,1)
   


    x_train = scaler_x.fit_transform(x_train)
    y_train = scaler_y.fit_transform(y_train)
    x_train_timestamp = scaler_time.fit_transform(x_train_timestamp)
    x_train_main = scaler_main.fit_transform(x_train_main)
    x_train_mean = scaler_mean.fit_transform(x_train_mean)
    x_train_std = scaler_std.fit_transform(x_train_std)
    x_train_max_min = scaler_max_min.fit_transform(x_train_max_min)
    x_train_max = scaler_max.fit_transform(x_train_max)
    x_train_min = scaler_min.fit_transform(x_train_min)
    x_train_kurtosis = scaler_kurtosis.fit_transform(x_train_kurtosis)
    x_train_skew = scaler_skew.fit_transform(x_train_skew)
    # x_train_diff = scaler_diff.fit_transform(x_train_diff)

    print(0)
    # test
    x_test = []
    y_test = []
    x_test_timestamp = []
    x_test_mean = []
    x_test_std = []
    x_test_max_min = []
    x_test_timestamp_true =[]
    x_test_main = []
    x_test_max = []
    x_test_min = []
    x_test_kurtosis = []
    x_test_skew = []

    for key, values in test.items():
        for app in range(len(appliances)):
            df = pd.read_csv(
                f"datasets/Building{key}_NILM_data_basic.csv", usecols=["Timestamp", "main", appliances[app]])
            df["date"] = pd.to_datetime(df["Timestamp"]).dt.date
            startDate = datetime.strptime(values["start_time"], "%Y-%m-%d").date()
            endDate = datetime.strptime(values["end_time"], "%Y-%m-%d").date()

            if startDate > endDate:
                raise "Start Date must be smaller than Enddate."

            df = df[(df["date"] >= startDate) & (df["date"] <= endDate)]
            df.dropna(inplace=True)
            # x = df["main"].values 

            if app == 0:
                x = df[appliances[app]].values
            else:
                print(app)
                x += df[appliances[app]].values
            if appliances[app] == "Refrigerator":
                y = df[appliances[app]].values
            # x = x - 100
            # x_diff = x
            # for i in range(1,len(x)):
            #     x_diff[i] = x_diff[i] - x_diff[i-1]
            # 
            # y = df[appliances[0]].values
        x_test_main.extend(x)
        timestamp_true = df["Timestamp"].values
        timestamp = (pd.to_datetime(df["Timestamp"]).astype(int)/ 10**18).values
        x2 = x
        x = jnp.pad(x, (units_to_pad, units_to_pad),
                    'constant', constant_values=(0, 0))
        x = jnp.array([x[i: i + n] for i in range(len(x) - n + 1)])

        x_test_mean.extend(jnp.mean(x, axis=1))
        x_test_std.extend(jnp.std(x, axis=1))
        x1 = jnp.pad(x2, (units_to_pad1, units_to_pad1),
                    'constant', constant_values=(0, 0))
        x1 = jnp.array([x1[i: i + m] for i in range(len(x1) - m + 1)])
        x_test_max_min.extend(jnp.max(x, axis=1)-jnp.min(x, axis=1))
        x_test_max.extend(jnp.max(x1, axis=1))
        x_test_min.extend(jnp.min(x1, axis=1))
        x_test.extend(x)
        x_test_kurtosis.extend(kurtosis(x,axis=1))
        x_test_skew.extend(skew(x,axis=1))
        y_test.extend(y)
        x_test_timestamp_true.extend(timestamp_true)
        x_test_timestamp.extend(timestamp)
        # x_test_diff.extend(x_diff)

    # for i in range(1,len(x_test)):
    #     x_test[i] = x_test[i] - x_test[i-1]
    x_test = jnp.array(x_test)
    y_test = jnp.array(y_test).reshape(-1, 1)
    x_test_timestamp = torch.tensor(x_test_timestamp).reshape(-1,1)
    x_test_main = jnp.array(x_test_main).reshape(-1,1)
    x_test_mean = jnp.array(x_test_mean).reshape(-1,1)
    x_test_std =  jnp.array(x_test_std).reshape(-1,1)
    x_test_max_min =  jnp.array(x_test_max_min).reshape(-1,1)
    x_test_max = jnp.array(x_test_max).reshape(-1,1)
    x_test_min = jnp.array(x_test_min).reshape(-1,1)
    x_test_kurtosis = jnp.array(x_test_kurtosis).reshape(-1,1)
    x_test_skew = jnp.array(x_test_skew).reshape(-1,1)

    x_test = scaler_x.transform(x_test)
    x_test_timestamp = scaler_time.transform(x_test_timestamp)
    x_test_mean = scaler_mean.transform(x_test_mean)
    x_test_std = scaler_std.transform(x_test_std)
    x_test_max_min = scaler_max_min.transform(x_test_max_min)
    x_test_main = scaler_main.transform(x_test_main)
    x_test_max =  scaler_max.transform(x_test_max)
    x_test_min = scaler_min.transform(x_test_min)
    x_test_kurtosis = scaler_kurtosis.transform(x_test_kurtosis)
    x_test_skew = scaler_skew.transform(x_test_skew)
    # x_test_diff = scaler_diff.transform(x_test_diff)
#     y_test = scaler_y.transform(y_test)

    # x_train_features = x_train_main_dif
    # x_test_features = x_test_main_dif

    x_train = jnp.array(x_train[:,1:]).reshape(x_train.shape[0], n-1)
    y_train = jnp.array(y_train)
    x_train_timestamp = torch.tensor(x_train_timestamp).reshape(x_train_timestamp.shape[0], 1)
    x_test = jnp.array(x_test[:,1:]).reshape(x_test.shape[0], n-1)
    y_test = jnp.array(y_test)
    x_test_timestamp = torch.tensor(x_test_timestamp).reshape(x_test_timestamp.shape[0], 1).to(torch.float64)

    #x_train_std, 
    #, x_test_std, x_test_max_min,
    num_features_selected = 6
    x_train_features = jnp.concatenate((x_train_main, x_train_mean,  x_train_max_min,x_train_max, x_train_min, x_train_kurtosis), axis=1).reshape(x_train.shape[0], num_features_selected)
    x_test_features = jnp.concatenate((x_test_main, x_test_mean,  x_test_max_min, x_test_max, x_test_min,  x_test_kurtosis), axis=1).reshape(x_test.shape[0], num_features_selected)

    scalers = np.array([scaler_x, scaler_y, scaler_time, scaler_main, scaler_mean, scaler_std, scaler_max_min, scaler_max, scaler_min, scaler_kurtosis, scaler_skew])
    return x_train, y_train, x_test, y_test, x_train_features, x_test_features, x_train_timestamp, x_test_timestamp, scalers, x_test_main


In [6]:
x_train, y_train, x_test, y_test, x_train_features, x_test_features, x_train_timstamp, x_test_timestamp, scalers, x_test_main = dataset_load(appliances, train, test)

1
2
1
2
0
1
2


In [7]:
x_train_diff = x_train_features[:,0]
diff = np.array(x_train_diff)
for i in range(1, len(x_train_diff)):
		value = x_train_diff[i] - x_train_diff[i - 1]
		diff[i] = value

x_test_diff = x_test_features[:,0]
diff = np.array(x_test_diff)
for i in range(1, len(x_test_diff)):
		value = x_test_diff[i] - x_test_diff[i - 1]
		diff[i] = value

# x_train_zeros = np.zeros(x_train_diff.shape)
# x_test_zeros = np.zeros(x_test_diff.shape)

In [8]:
x_train_features = jnp.concatenate((x_train_features, jnp.array(np.array(x_train_diff.reshape(-1,1)))), axis=1)
x_test_features = jnp.concatenate((x_test_features, jnp.array(np.array(x_test_diff.reshape(-1,1)))), axis=1)

#, jnp.array(x_test_timestamp)

In [40]:
x_train_features.shape, x_test_features.shape

((24918, 7), (15656, 7))

In [10]:
import gpytorch
indexes = 53456
n = x_train_features.shape[1]
x = torch.tensor(np.array(x_train_features)).to(torch.float64)
y = torch.tensor(np.array(y_train)).reshape(-1,).to(torch.float64)
xt = torch.tensor(np.array(x_test_features)).to(torch.float64)
yt = torch.tensor(np.array(y_test)).reshape(-1,).to(torch.float64)

if x.shape[0]>indexes:
  x = x[:indexes]
  y = y[:indexes]

# xt = xt[:16000]
# yt = yt[:16000]

In [41]:
xt.shape

torch.Size([15656, 7, 1])

In [61]:
import torch.nn as nn
class seq2point(torch.nn.Module):
    def __init__(self):
        super(seq2point, self).__init__()
        self.c1 = nn.Conv1d(7,7, kernel_size=1)
        self.r1 = nn.ReLU()
        self.c2 = nn.Conv1d(7,3,kernel_size=1)
        self.r2 = nn.ReLU()        
        self.c3 = nn.Conv1d(3,1, kernel_size=1)
        self.r3 = nn.ReLU()
        # c4 = nn.Conv(50, kernel_size=(5,))(X)
        # X = nn.ReLU(X)
        # X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        # X = nn.Conv(50, kernel_size=(5,))(X)
        # X = nn.ReLU(X)
        # X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        # X = X.reshape((X.shape[0], -1))
        # X = nn.Dense(1024)(X)
        # X = nn.ReLU(X)
        # X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        self.d = nn.Linear(1,1)

    def forward(self,X):
        print(f"foawrd is running")
        X = self.c1(X)
        print(f"shape of X1 -> {X.shape}")

        X = self.r1(X)
        print(f"shape of X2 -> {X.shape}")

        X = self.c2(X)
        print(f"shape of X3 -> {X.shape}")

        X = self.r2(X) 
        print(f"shape of X4 -> {X.shape}")

        X = self.c3(X)
        print(f"shape of X5 -> {X.shape}")

        X = self.r3(X)
        print(f"shape of X6 -> {X.shape}")
        # X = nn.Conv(50, kernel_size=(5,))(X)
        # X = nn.ReLU(X)
        # X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        # X = nn.Conv(50, kernel_size=(5,))(X)
        # X = nn.ReLU(X)
        # X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        # X = X.reshape((X.shape[0], -1))
        # X = nn.Dense(1024)(X)
        # X = nn.ReLU(X)
        # X = nn.Dropout(rate=0.2, deterministic=deterministic)(X)
        X = self.d(X)
        print(f"final shape -> {X.shape}")
        return X

feature_extractor1 = seq2point()


In [76]:
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            print("GPRegressionModel initialized")
            self.mean_module = gpytorch.means.ConstantMean()
            print("mean instantiated")
            self.base_covar_module = ( gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5,ard_num_dims=n)))
            print("covariance insitantiated")
            #+ gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=1, active_dims=(49)))*gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel(ard_num_dims=1, active_dims=(49)))
            # self.covar_module = gpytorch.kernels.GridInterpolationKernel(
            #     gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2)),
            #     num_dims=2, grid_size=100
            # )
            self.likelihood = likelihood
            # self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=train_x[np.arange(0, train_x.shape[0], 95)], likelihood=likelihood)
            # print(self.covar_module)
            self.feature_extractor1 = feature_extractor1
            # self.feature_extractor2 = feature_extractor2

            # This module will scale the NN features so that they're nice values
            # self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x1):
            print("GPRegression forward is starting")
            # We're first putting our data through a deep net (feature extractor)
            projected_x = self.feature_extractor1(x1)
            print("Testing if feature extractor works")
            print(projected_x.shape)
            # print(projected_x.shape, hn.shape,cn.shape)
            # projected_x = self.feature_extractor2(projected_x)
            # print(projected_x.shape, hn.shape,cn.shape)
            # projected_x = self.scale_to_bounds(projected_x)  # Make the NN values "nice"
            # project_x = projected_x
            # plt.figure()
            # plt.plot(projected_x.detach().cpu()[:,49],x1.detach().cpu()[:,49])
            projected_x = projected_x - projected_x.min(0)[0]
            projected_x = 2 * (projected_x / projected_x.max(0)[0]) - 1
            print(projected_x.dtype)
            projected_x = projected_x.squeeze(-1)
            print(f"new x shape -> {projected_x.shape}")
            self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=projected_x[np.arange(0, projected_x.shape[0], 95)], likelihood=self.likelihood)
            print("covariance matrix sampled")
            mean_x = self.mean_module(projected_x) #projected_
            print("mean calculated")
           
            covar_x = self.covar_module(projected_x)
            print("covariance matrix generated")
            
            ret_val =  gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
            print(f"distribution formed")
            print(ret_val)
            return ret_val
        

In [77]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(x, y, likelihood)

# if torch.cuda.is_available():
#     model = model.cuda()
#     likelihood = likelihood.cuda()

GPRegressionModel initialized
mean instantiated
covariance insitantiated


In [78]:
x = x.reshape(24918,7,1)
x.shape
x = torch.tensor(x)

# x_try = torch.randn(64,7,100)
trial = feature_extractor1(x)
trial.shape


foawrd is running
shape of X1 -> torch.Size([24918, 7, 1])
shape of X2 -> torch.Size([24918, 7, 1])
shape of X3 -> torch.Size([24918, 3, 1])
shape of X4 -> torch.Size([24918, 3, 1])
shape of X5 -> torch.Size([24918, 1, 1])
shape of X6 -> torch.Size([24918, 1, 1])
final shape -> torch.Size([24918, 1, 1])


/tmp/ipykernel_2248320/1843471022.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)


torch.Size([24918, 1, 1])

In [79]:
import tqdm
training_iterations = 2

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
loss_arr =[]
def train():
    iterator = tqdm.notebook.tqdm(range(training_iterations))
    for i in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output= model(x.to(torch.float32))
        # print(output.loc.shape)
        # Calc loss and backprop derivatives
        print(0)
        loss = -mll(output, y.to(torch.float32))
        # loss_arr.append(loss.cpu())
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        optimizer.step()
 
%time train()

  0%|          | 0/2 [00:00<?, ?it/s]

GPRegression forward is starting
foawrd is running
shape of X1 -> torch.Size([24918, 7, 1])
shape of X2 -> torch.Size([24918, 7, 1])
shape of X3 -> torch.Size([24918, 3, 1])
shape of X4 -> torch.Size([24918, 3, 1])
shape of X5 -> torch.Size([24918, 1, 1])
shape of X6 -> torch.Size([24918, 1, 1])
final shape -> torch.Size([24918, 1, 1])
Testing if feature extractor works
torch.Size([24918, 1, 1])
torch.float32
new x shape -> torch.Size([24918, 1])
covariance matrix sampled
mean calculated
covariance matrix generated
distribution formed
MultivariateNormal(loc: torch.Size([24918]))
0


/home/desai.aadesh/.local/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/desai.aadesh/.local/lib/python3.8/site-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(


GPRegression forward is starting
foawrd is running
shape of X1 -> torch.Size([24918, 7, 1])
shape of X2 -> torch.Size([24918, 7, 1])
shape of X3 -> torch.Size([24918, 3, 1])
shape of X4 -> torch.Size([24918, 3, 1])
shape of X5 -> torch.Size([24918, 1, 1])
shape of X6 -> torch.Size([24918, 1, 1])
final shape -> torch.Size([24918, 1, 1])
Testing if feature extractor works
torch.Size([24918, 1, 1])
torch.float32
new x shape -> torch.Size([24918, 1])
covariance matrix sampled
mean calculated
covariance matrix generated
distribution formed
MultivariateNormal(loc: torch.Size([24918]))
0
CPU times: user 49.1 s, sys: 525 ms, total: 49.7 s
Wall time: 940 ms


In [31]:
model_name = "deep_kernel.pt"
torch.save(model.state_dict(), os.path.join(
  "Neurips/models", model_name))

In [71]:
# xt = xt.squeeze(-1)
# xt.shape


In [72]:
# # testing 
# try_x = torch.randn(15656, 7, 1)
# new_model = seq2point()
# print(new_model(try_x).shape)
# preds = model(try_x)


In [81]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    xt_new = xt.reshape(-1,7,1).to(torch.float32)
    # print(xt_new.shape)
    preds= model(xt_new)

torch.Size([15656, 7, 1])


In [ ]:
y_mean = np.array(preds.mean.cpu())
y_mean = scalers[1].inverse_transform(y_mean.reshape(-1,1)).squeeze()
y_mean2 = np.clip(y_mean,0,y_mean.max(),out=y_mean)

In [ ]:
print('Test MAE: {}'.format(torch.mean(torch.abs(torch.tensor(y_mean2) - yt))))

In [ ]:
time_plot = scalers[2].inverse_transform(x_test_timestamp.reshape(-1,1))
x1 = x_test_features[:,0].reshape(-1,1)

In [ ]:
idx = 1000 #x2.shape[0]
start = 4000
plt.figure(figsize=(10,6))
# plt.plot(time_plot[start:start+idx], scalers[3].inverse_transform(x1)[start:start+idx], label = "Main")
# plt.plot(x_test_timestamp[:idx], scalers[-1].inverse_transform(x2)[:idx], label = "Main Ref Dif")
plt.plot(time_plot[start:start+idx], yt.cpu()[start:start+idx], label = "Refri")
plt.plot(time_plot[start:start+idx], y_mean[start:start+idx], label = "Predicted")
plt.legend(bbox_to_anchor=(1, 1),fontsize=20)
plt.xlabel("TimeStamp test Scaled")
plt.ylabel("Fridge  test Power")
plt.title("Building 6")
sns.despine()